In [1]:
import os
import torch

from face_recognition.models.FaceNetPytorchLightning import LightningFaceNet

In [2]:
results_path = './face_recognition/results/models'

In [3]:
checkpoint = './face_recognition/checkpoints/last_checkpoint-v8.ckpt'

In [4]:
import pytorch_lightning as pl

from torchvision import transforms
from face_recognition.data.datasets import LFWValidationDataset

In [5]:
root = os.path.join('face_recognition', 'data', 'images', 'lfw_aligned')
pairs_txt = os.path.join('face_recognition', 'data', 'images', 'pairs.txt')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
lfw_set = LFWValidationDataset(root, pairs_txt, transform=transform)

In [6]:
from face_recognition.models.FaceNet import FaceNetResnet

In [7]:
hparams = {'margin': 0.02, 'optimizer': 'adagrad', 'weight_decay': 1e-5, 'lr': 0.05}
backbone = FaceNetResnet(pretrained=True)
model = LightningFaceNet(hparams, backbone)

In [8]:
model = LightningFaceNet.load_from_checkpoint(checkpoint, hparams=hparams, model=backbone)

In [9]:
model

LightningFaceNet(
  (model): FaceNetResnet(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): Re

In [10]:
trainer = pl.Trainer(
    gpus=1 if torch.cuda.is_available() else 0,
    max_epochs=3,
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [11]:
from torch.utils.data import DataLoader

test_loader = DataLoader(lfw_set, batch_size=16, shuffle=True, num_workers=8)
trainer.test(model, test_loader)

Testing:   0%|                                                                                                                                                                                                                  | 0/375 [00:00<?, ?it/s]

c:\users\thien\projects\ibm-labcourse\.venv\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:59<00:00,  6.31it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'fp_rate': 0.7683597417564018,
 'precision': 0.8950968862330008,
 'recall': 0.8647685236631848,
 'roc_auc': 0.947035185323572,
 'test_acc': 0.8821666666666665,
 'tp_rate': 0.8767592453679406}
--------------------------------------------------------------------------------


[{'tp_rate': 0.8767592453679406,
  'fp_rate': 0.7683597417564018,
  'precision': 0.8950968862330008,
  'recall': 0.8647685236631848,
  'roc_auc': 0.947035185323572,
  'test_acc': 0.8821666666666665}]

In [12]:
model_name = os.path.join(results_path, 'presentation_model.pth')

In [13]:
torch.save(model.model.state_dict(), model_name)

In [8]:
from face_recognition.models.FaceNet import load_weights

In [9]:
model = load_weights(model_name)


In [10]:
model

FaceNetResnet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [16]:
enc = model(lfw_set[0][0].unsqueeze(0))

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 128])

In [ ]:
enc.shape